<h1> Script predicción</h1> 
<h2> The Backpropagation Boyz </h2>

# Librerias y funciones  

In [1]:
import numpy as np 
import os 
import tensorflow as tf
import sklearn 
import matplotlib.pyplot as plt
import pandas as pd
from scipy import ndimage
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#clustering
from scipy.cluster.hierarchy import complete,single, fcluster,dendrogram
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import pdist
from sklearn.metrics import pairwise_distances
import datetime
import seaborn as sns




#esta funcion coge series como input y las secciona, dando n puntos iniciales (feature) y los m siguientes (labels)

def window(data_big,size_input,size_output=None,data_aux=None):
  if size_output is not None:
    #esto para implementar que se devuelvan ventanas mas grandes
    features=[]
    labels=[]
    #if data_aux is not None:
      #data_aux=data_aux.T

    for i in range(0,data_big.shape[0]):
      data=data_big[i]
      for j in range(data_big.shape[1] - size_input -size_output):
        if data_aux is not None:
          data_aux_slice=data_aux[j:(j+size_input),:]
          features.append(np.hstack([data[j:(j+size_input)][...,np.newaxis],data_aux_slice]))
        else:
          features.append(data[j:(j+size_input)])
        labels.append(data[(j+size_input):(j+size_input+size_output)])
  else:
    features=[]
    labels=[]
    for i in range(0,data_big.shape[0]):
      data=data_big[i]
      for j in range(data_big.shape[1] - size_input -1):
        if data_aux is not None:
          data_aux_slice=data_aux[j:(j+size_input),:]
          features.append(np.hstack([data[j:(j+size_input)][...,np.newaxis],data_aux_slice]))
        else:
          features.append(data[j:(j+size_input)])
        labels.append(data[j+size_input+1])
  return np.array(features),np.array(labels)  



#funcion que coge una serie temporal, un modelo y va predicciendo todo lo que puede 
#si toma inputs de 20 y devuelve 7, va prediciendo asi la serie temporal (de 7 en en 7 conociendo los 20 anteriores)
def pred_model2(model,ts,window):
  input_size=model.input_shape[1]
  leng=ts.shape[0]
  preds=[]
  for i in range(input_size,leng-window,window):
    preds.append(model.predict(ts[(i-input_size):i][np.newaxis,...])[0])
  return np.array(preds)

# Carga y preprocesado de datos

Comenzamos ordenando las muestras del fichero.

In [2]:
!sort Modelar_UH2022.txt > Modelar_UH2022_sorted.txt

In [3]:
!head Modelar_UH2022_sorted.txt

0|2019-02-01 00:39:36|331710|0|0|0
0|2019-02-01 01:39:36|331710|0|0|0
0|2019-02-01 02:39:35|331710|0|0|0
0|2019-02-01 03:39:35|331710|0|0|0
0|2019-02-01 04:39:35|331710|0|0|0
0|2019-02-01 05:39:35|331710|0|0|0
0|2019-02-01 06:39:35|331710|0|0|0
0|2019-02-01 07:39:35|331729|0|19|0
0|2019-02-01 08:39:35|331765|0|36|0
0|2019-02-01 09:39:35|331765|0|0|0


La cabecera se ha quedado al final, vamos a moverla al principio.

In [4]:
!sed '1h;1d;$!H;$!d;G' Modelar_UH2022_sorted.txt > tmp.txt; mv tmp.txt Modelar_UH2022_sorted.txt

In [5]:
!head Modelar_UH2022_sorted.txt

ID|SAMPLETIME|READINGINTEGER|READINGTHOUSANDTH|DELTAINTEGER|DELTATHOUSANDTH
0|2019-02-01 00:39:36|331710|0|0|0
0|2019-02-01 01:39:36|331710|0|0|0
0|2019-02-01 02:39:35|331710|0|0|0
0|2019-02-01 03:39:35|331710|0|0|0
0|2019-02-01 04:39:35|331710|0|0|0
0|2019-02-01 05:39:35|331710|0|0|0
0|2019-02-01 06:39:35|331710|0|0|0
0|2019-02-01 07:39:35|331729|0|19|0
0|2019-02-01 08:39:35|331765|0|36|0


No nos interesan los datos de cada hora, vamos a agregarlos para tener la información diaria de cada deposito. Asimismo, tendremos en cuenta un solo valor sin decimales.

In [6]:
!rm Modelar_UH2022_agg.txt

rm: cannot remove 'Modelar_UH2022_agg.txt': No such file or directory


In [7]:
file1 = open('Modelar_UH2022_sorted.txt', 'r')
file2 = open('Modelar_UH2022_agg.txt', 'a')
iden_pre = ""
curr_delta = 0
curr_reading = 0

for i, line in enumerate(file1.readlines()):
    if i != 0:
        iden, content  = line.split()
        content = content.split('|')

        if '' in content: content = ['0' if c == '' else c for c in content]

        if iden != iden_pre and i!= 1:

            file2.write(iden_pre+'|'+ str(curr_reading)+'|'+str(curr_delta)+'\n')
            curr_delta = 0

            
        curr_reading = int(content[1])*100 + int(content[2])*(10**(2-len(content[2])))

        curr_delta += int(content[3])*100 + int(content[4])*(10**(2-len(content[2])))

        iden_pre = iden

    else:
        file2.write('ID|SAMPLETIME|READING|DELTA\n')

file1.close()
file2.close()

In [8]:
!rm Modelar_UH2022_sorted.txt

In [9]:
datos = pd.read_csv("Modelar_UH2022_agg.txt", sep='|')
print(datos.head())

   ID  SAMPLETIME   READING  DELTA
0   0  2019-02-01  33195300  24300
1   0  2019-02-02  33218900  23600
2   0  2019-02-03  33252400  33500
3   0  2019-02-04  33277600  25200
4   0  2019-02-05  33299600  22000


In [23]:
lectura = datos.iloc[:,1:].fillna(0,inplace=False).to_numpy().T

#vamos a suavizar todo y a escalarlo
data_filtered = ndimage.gaussian_filter(lectura,[0,3],0)
scaler=MinMaxScaler()
data_pre=scaler.fit_transform(data_filtered.T).T

RuntimeError: array type dtype('O') not supported